In [1]:
import os
import wfdb
import numpy as np
import pandas as pd

In [2]:
# DOWNLOAD RECORD INFORMATION FILES
wfdb.dl_files('chbmit',dl_dir='record_info',
              files=['RECORDS','RECORDS-WITH-SEIZURES'])

Created local base download directory: record_info
Finished downloading files


In [3]:
# Get channels, start-end, and seizure start-end for all files in subject
def get_sub_info(file):
    subject = [i[:-1] for i in open(file, 'r')]
    file_chans = {}
    start_end = {}
    seiz_start_end = {}
    collect_seiz_time = False
    for i in subject:
        if 'Channel 1:' in i:
            collect_chan = True
            channel_group = {}
        if 'Channel' not in i:
            collect_chan = False
        if collect_chan:
            channel_group[i.split(':')[0]] = i.split(':')[1].strip()
        if 'File Name:' in i:
            f_name = i.split(':')[1].strip()
            file_chans[f_name] = channel_group
            start_end[f_name] = []
        if i.split(':')[0]=='Number of Seizures in File' \
        and float(i.split(':')[1].strip())>0:
            s_e = []
            seiz_start_end[f_name] = []
        if ('Seizure' in i and 'Start' in i):
            collect_seiz_time = True
        if i=='':
            collect_seiz_time = False
        if collect_seiz_time:
            s_e.append(float(i.split(':')[1].strip().split(' ')[0]))
            seiz_start_end[f_name] = s_e
        if 'File Start Time' in i:
            s_time = i.split(': ')[1]
        if 'File End Time' in i:
            start_end[f_name] = [s_time,i.split(': ')[1]]
    return file_chans, start_end, seiz_start_end

In [4]:
# Get and combine file_chans, start_end, and seiz_start_end for all subjects
sub_files = ['record_info/' + i for i in \
             os.listdir('record_info') if 'summary' in i]
file_chans = {}
file_s_e = {}
file_seize_s_e = {}

for i in sub_files:
    fc, se, sse = get_sub_info(i)
    file_chans.update(fc)
    file_s_e.update(se)
    file_seize_s_e.update(sse)

In [9]:
sub_files

[]

In [5]:
# Get list of seizure files
seizure_records = [i[:-1] for i in \
                   open('record_info/RECORDS-WITH-SEIZURES','r') \
                   if i[:-1] != '']
seizure_files = [i for i in file_chans.keys() \
                 if i[:5]+'/'+i in seizure_records]
seizure_electrodes = [file_chans[i].values() for i in seizure_files]
matched_electrodes = [i for i in range(len(seizure_electrodes)) \
                      if len(set(seizure_electrodes[0]).intersection\
                             (seizure_electrodes[i]))==22]
seizures = seizure_files[matched_electrodes]

TypeError: list indices must be integers or slices, not list